In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# % cd /content/drive/MyDrive/xa/code_xa/project3

In [ ]:
# %%capture 
# ! pip install -r requirements.txt

In [ ]:
%%capture
! pip install icecream
from icecream import ic

In [ ]:
# ! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
# /content/drive/MyDrive/xa/code_xa/project3/yolov5/train.py

Train Yolov5 in custom data
---

In [ ]:
%cd /content/drive/MyDrive/xa/code_xa/project3
!ls

In [ ]:
%%capture
! pip install -r /content/drive/MyDrive/xa/code_xa/project3/yolov5/requirements.txt

In [ ]:
# !python yolov5/train.py --img 640 --batch 70 --epochs 100  --data yolov5/data/table_sami.yaml --weights yolov5s.pt --cache

weight lưu ở exp29

In [ ]:
%cd /content/drive/MyDrive/xa/code_xa/project3/yolov5


In [ ]:
# !python detect.py --weights /content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/train/exp21/weights/last.pt --source ../project3_data_test --save-txt  

In [ ]:
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [ ]:
def plot(img):
  plt.figure(figsize=(15,10))
  plt.imshow(img, cmap = "gray")
  plt.show()

# Xoay ảnh 

In [ ]:
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import rotate
from skimage.feature import canny
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from scipy.stats import mode

image = rgb2gray(imread("/content/drive/MyDrive/xa/code_xa/project3/project3_data_test/Copy of BĐ ĐS-MI1112 - GK2020.1-63.jpg"))

edges = canny(image)
# Classic straight-line Hough transform
tested_angles = np.deg2rad(np.arange(0.1, 180.0))
h, theta, d = hough_line(edges, theta=tested_angles)

# Generating figure 1
fig, axes = plt.subplots(1, 2, figsize=(15, 16))
ax = axes.ravel()

ax[0].imshow(image, cmap="gray")
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(edges, cmap="gray")
origin = np.array((0, image.shape[1]))

for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
    ax[1].plot(origin, (y0, y1), '-r')
    
ax[1].set_xlim(origin)
ax[1].set_ylim((edges.shape[0], 0))
ax[1].set_axis_off()
ax[1].set_title('Detected lines')

In [ ]:
import cv2
def skew_angle_hough_transform(image):
    # convert to edges
    edges = canny(image)
    # Classic straight-line Hough transform between 0.1 - 180 degrees.
    tested_angles = np.deg2rad(np.arange(0.1, 180.0))
    h, theta, d = hough_line(edges, theta=tested_angles)
    
    # find line peaks and angles
    accum, angles, dists = hough_line_peaks(h, theta, d)
    
    # round the angles to 2 decimal places and find the most common angle.
    most_common_angle = mode(np.around(angles, decimals=2))[0]
    
    # convert the angle to degree for rotation.
    skew_angle = np.rad2deg(most_common_angle - np.pi/2)
    print(skew_angle)
    return skew_angle
    
fig, ax = plt.subplots(ncols=2, figsize=(20,20))
ax[0].imshow(image, cmap="gray")
rotated_image = rotate(image, skew_angle_hough_transform(image),  cval=1)
ax[1].imshow(rotated_image, cmap = "gray")
print(type(rotated_image))
from PIL import Image
I = rotated_image
I8 = (((I - I.min()) / (I.max() - I.min())) * 255.9).astype(np.uint8)

img = Image.fromarray(I8)
img.save("/content/drive/MyDrive/xa/code_xa/project3/project3_data_rotated/Copy of BĐ ĐS-MI1112 - GK2020.1-63.jpg")
plt.show()

# Sử dụng yoloV5 để xác định bảng điểm 

In [ ]:
# !python detect.py --weights /content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/train/exp21/weights/last.pt --source ../project3_data_rotated --save-txt  


In [ ]:
%cd /content/drive/MyDrive/xa/code_xa/project3/yolov5

In [ ]:

import argparse
import os
import sys
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

ROOT = "/content/drive/MyDrive/xa/code_xa/project3/yolov5"
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative
@torch.no_grad()
def detect_by_yolov5(weights=ROOT / 'yolov5s.pt',  # model.pt path(s)
        source=ROOT / 'data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        ):
    all_detect = []
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn)
    stride, names, pt, jit, onnx = model.stride, model.names, model.pt, model.jit, model.onnx
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Half
    half &= pt and device.type != 'cpu'  # half precision only supported by PyTorch on CUDA
    if pt:
        model.model.half() if half else model.model.float()

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt and not jit)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt and not jit)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    if pt and device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.model.parameters())))  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3
        


        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                     # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        ic(xywh)
                        
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        detected =  (('%g ' * len(line)).rstrip() % line + '\n')
                        all_detect.append(detected)
    return all_detect
# detect_by_yolov5(weights = weight_yolov5, device = "cuda", source = path )

## Hiện ảnh

In [ ]:
import PIL 
path_img = "/content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/detect/exp10/Copy of BĐ ĐS-MI1112 - GK2020.1-63.jpg"
image_test = PIL.Image.open(path_img)
# image to open
image_width, image_height = image_test.size
image_test = cv2.imread(path_img)
plot(image_test)

## Lấy kết quả từ yolov5

In [ ]:
path_source_img = "/content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/detect/exp10/Copy of BĐ ĐS-MI1112 - GK2020.1-63.jpg"
weight_yolov5 = "/content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/train/exp21/weights/last.pt"
yolov5_detected =  detect_by_yolov5(weights = weight_yolov5, device = "cuda", source = path_source_img )

In [ ]:
yolov5_detected

In [ ]:
# path_root_result = "/content/drive/MyDrive/xa/code_xa/project3/yolov5/runs/detect/exp10/labels"
# result = path_img.split("/")[-1]
# path_result = path_root_result + "/" + result
# path_result = path_result.replace(".jpg",".txt")

In [ ]:
# with open(path_result,"r") as f:
#   # print(f.readline())
_, x_center, y_center,width, height = [float(x) for x in yolov5_detected[0].split()]

In [ ]:
print( _, x_center, y_center, width, height)
      # x0, y0, x1, y1 = bbox


In [ ]:
print(image_width, image_height)

In [ ]:
bbox = [ (x_center-width/2)*image_width, (y_center-height/2)*image_height, width*image_width, height*image_height  ]
bbox = [int(x) for x in bbox]
bbox

In [ ]:
plot(image_test[  bbox[1]:bbox[1]+bbox[3],  bbox[0]: bbox[0]+bbox[2]  ])

In [ ]:
#  ###################################

In [ ]:
# img = cv2.imread('/content/drive/MyDrive/xa/code_xa/project3/Bd_crop.jpg')
# img
img = image_test[  bbox[1]:bbox[1]+bbox[3],  bbox[0]: bbox[0]+bbox[2]  ]

In [ ]:
print(bbox[1], bbox[1]+bbox[3],  bbox[0] ,  bbox[0]+bbox[2])
#367 2102 103 1614

In [ ]:
type(img)

In [ ]:
def preprocess(img, factor: int):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = Image.fromarray(img)
    enhancer = ImageEnhance.Sharpness(img).enhance(factor)
    if gray.std() < 30:
        enhancer = ImageEnhance.Contrast(enhancer).enhance(factor)
    return np.array(enhancer)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(24, 8))
img_process =  preprocess(img, 30)
plt.figure(figsize=(15,10))
plt.imshow(img)
plt.figure(figsize=(15,10))
plt.imshow(img_process)
plt.show()

biến đổi thành nhị phân

In [ ]:
gray_scale=cv2.cvtColor(img_process,cv2.COLOR_BGR2GRAY)
th1,img_bin = cv2.threshold(gray_scale,150,225,cv2.THRESH_BINARY)
img_bin=~img_bin
plot(img_bin)


Applying Morphological Operations

In [ ]:
### selecting min size as 15 pixels
line_min_width = 15
kernal_h = np.ones((1,line_min_width), np.uint8)
kernal_v = np.ones((line_min_width,1), np.uint8)

Applying Horizontal Kernel on the image

In [ ]:
img_bin_h = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_h)

Applying Vertical Kernel on the image

In [ ]:
img_bin_v = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_v)
img_bin_v

Merging Horizontal and Vertical Images

In [ ]:
img_bin_final=img_bin_h|img_bin_v

Now, as you can see from the above image, we can filter most of the text as noise, we do a Connected Component Analysis on the image to get the Bounding Boxes of the checkboxes.

In [ ]:
_, labels, stats,_ = cv2.connectedComponentsWithStats(~img_bin_final, connectivity= 4, ltype=cv2.CV_32S)

`stats`:   
```  
Combine vertical and horizontal
        Args: 
            bw: black white image
        Return:
            num_labels: (int) number component after connect '1' element in image
            stats: (array-[num_labels, 5]) [left, top, width, height, area] of component
            centroids:  (array-[num_labels, 2]) [x, y] centroid of component  
```

In [ ]:
for x,y,w,h,area in stats[2:]:
    cv2.rectangle(img_process,(x,y),(x+w,y+h),(0,255,0),2)

In [ ]:
# kernel_len = gray.shape[1]//120
# hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
# image_horizontal = cv2.erode(img_bin, hor_kernel, iterations=3)
# horizontal_lines = cv2.dilate(image_horizontal, hor_kernel, iterations=3)

# h_lines = cv2.HoughLinesP(
#     horizontal_lines, 1, np.pi/180, 30, maxLineGap=250)
# plt.figure(figsize=(15,10))
# plt.imshow(h_lines)
# plt.imshow(h_lines)

# plt.show()

In [ ]:
def detect_box(image,line_min_width=15):
    
    gray_scale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    th1,img_bin=cv2.threshold(gray_scale,150,230,cv2.THRESH_BINARY)
    kernal_h=np.ones((1,line_min_width), np.uint8)
    kernal_v=np.ones((line_min_width,1), np.uint8)
    img_bin_h=cv2.morphologyEx(~img_bin, cv2.MORPH_OPEN, kernal_h)
    img_bin_v=cv2.morphologyEx(~img_bin, cv2.MORPH_OPEN, kernal_v)
    
    img_bin_final=img_bin_h|img_bin_v
    
    final_kernel=np.ones((3,3), np.uint8)
    img_bin_final=cv2.dilate(img_bin_final,final_kernel,iterations=1)
    ret, labels, stats,centroids = cv2.connectedComponentsWithStats(~img_bin_final, connectivity=4, ltype=cv2.CV_32S)
    return stats,labels

In [ ]:
### lets visualise the connected component image using the function

def imshow_components(labels):
    ### creating a hsv image, with a unique hue value for each label
    label_hue = np.uint8(179*labels/np.max(labels))
    ### making saturation and volume to be 255
    empty_channel = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, empty_channel, empty_channel])
    ### converting the hsv image to BGR image
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue==0] = 0
    ### returning the color image for visualising Connected Componenets
    return labeled_img

In [ ]:
stats,labels=detect_box(img_process)
cc_out=imshow_components(labels)

# lưu lại các tọa độ và cho nó vào hình 
for x,y,w,h,area in stats:
    cv2.rectangle(img_process,(x,y),(x+w,y+h),(0,255,0), 1)

plot(cc_out)
plot(img_process) 

In [ ]:
print(stats)

In [ ]:
stats.shape

In [ ]:
len(stats)

In [ ]:
stats_word = []
stats_hand = []

stt_list = []
mssv_list = []
ho_va_ten_list = []
lop_list = []
diem_list = []



# x y w h
for stat in stats :
  if  (10 < stat[2]  < 500) and ( 20 < stat[3] < 50):
    if (stat[0] < 50) and ( 10 < stat[2] < 100) and  (10 < stat[3] < 80) :
      stt_list.append(np.array([stat[0] + 3, stat[1]    , stat[2]-2, stat[3]  ]))
      continue
    if (20 < stat[0] < 400) and ( 10 < stat[2] < 300) and (10 < stat[3] < 80) :
      mssv_list.append(stat)
      continue

    if ( 150 < stat[0] < 500) and ( 100 < stat[2] < 450) and  (10 < stat[3] < 80) :
      ho_va_ten_list.append(stat)
      continue

    if (150< stat[0] < 700) and ( 100 < stat[2] < 500) and (10 < stat[3] < 80) :
      lop_list.append(stat)
      continue

    if (500 < stat[0] < 1100) and ( 10 < stat[2] < 200) and (stat[3] < 80) :
    #   try:
    #     diem_list.append(np.array([stat[0], stat[1]  - 10   , stat[2], stat[3]+  20   ]))
    #   except:

        # try:
        #     diem_list.append(np.array([stat[0], stat[1]  - 6   , stat[2], stat[3]+  15   ]))
        # except:
            try:
                diem_list.append(np.array([stat[0], stat[1] - 2 , stat[2], stat[3] + 1]))
                
            except:
                diem_list.append(np.array([stat[0], stat[1], stat[2], stat[3]]))
                continue

len(stats_word)
len(stats_hand)


In [ ]:
np.array([stats[2][0], stats[2][1], stats[2][2], stats[2][3]])

In [ ]:
img_draw = img.copy()

In [ ]:
ic(len(stt_list))
ic(len(mssv_list))
ic(len(ho_va_ten_list))
ic(len(lop_list))
ic(len(diem_list))


In [ ]:
stats_word = []
for i in range(len(stt_list)):
  stats_word.append(stt_list[i])
  stats_word.append(mssv_list[i])
  stats_word.append(ho_va_ten_list[i])
  stats_word.append(lop_list[i])
  stats_word.append(diem_list[i])




In [ ]:
img_crop_list = []
for i in  range(len(stats_word)):
  crop =  stats_word[i]
  img_crop = img[crop[1] : crop[1]+crop[3], crop[0]:crop[2]+crop[0]]
  img_crop_list.append(img_crop)
# for i in range(len(img_crop_list)):
  print("##########################___"+ str(i)+"________" + str(crop[0]) + "___" + str(crop[1]) + "___" + str(crop[2]) + "___" + str(crop[3]))
  plot(img_crop)
  


  # plot(img_crop_list[i])

In [ ]:
# stats_sort

In [ ]:
plot(img_crop_list[54])

**Model**

---


In [ ]:
%%capture 
!pip install craft-text-detector vietocr


In [ ]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from craft_text_detector import Craft

In [ ]:
!scp -r /content/drive/MyDrive/xa/code_xa/project3/weight_mnist/craft_mlt_25k.pth /root/.craft_text_detector/weights/

In [ ]:
config = Cfg.load_config_from_name('vgg_seq2seq')
config['device'] = 'cuda'
config['cnn']['pretrained'] = False
config['predictor']['beamsearch'] = False

craft = Craft(crop_type='box', cuda=True, text_threshold=0.6, \
        link_threshold=0.3, low_text=0.3, long_size=1280)
detector = Predictor(config)


In [ ]:
# img = Image.fromarray(img_box)
# pred = detector.predict(img)
# # pred = detector.predict(img_box)
# pred

In [ ]:
# text_boxes

In [ ]:
#@title Default title text
def boundingRect(bbox):
      bbox = bbox.astype(np.int)
      x0, y0 = np.min(bbox, axis=0)
      x1, y1 = np.max(bbox, axis=0)
      return x0, y0, x1, y1

def get_info_word(img_box):
  result_of_OCR = ""
  # img_box = img_crop_list[32]
  text_boxes = craft.detect_text(image=img_box)['boxes']
  if len(text_boxes) > 0:
      text_boxes = [boundingRect(bbox) for bbox in text_boxes]
      text_boxes = sorted(text_boxes, key=lambda x: (x[1], x[0]))
  s = []
  legit_box = []
  list_detect = []
  dict_of_OCR = {}
  dict_of_det = {}
  for bbox in text_boxes:
      x0, y0, x1, y1 = bbox
      if x0 < 0 or y0 < 0:
          continue
      try:
        img = img_box[y0 - 10: y1 + 10, x0  - 6: x1 + 6]
        # plot(img)
        img = Image.fromarray(img)
        
      except:
        img = img_box[y0: y1, x0: x1]
        img = Image.fromarray(img)


      list_detect.append(img)
      pred = detector.predict(img)
      # Draw bbox and name label
      # cv2.rectangle(img_box.copy(), (x + x0 - 5, y + y0 - 5), (x + x1 + 5,  y + y1 + 5), (0, 255, 0), 2)
      # cv2.putText(img_box.copy(), str(i), (x+x0-5, y+y0-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 3)
      # plot()
      # print(pred)
     
      if pred:
          s.append(pred)
          bbox = (x + x0, y + y0, x + x1, y + y1)
          dict_of_det[bbox] = pred
          
          # if 'Tên ' in pred: # mini trick
          #     ten_bigbox = stats[i]
  if s:
      result_of_OCR = " ".join(s)
  return result_of_OCR

def get_info_hand(img_box):
    result_of_OCR = ""
    img = Image.fromarray(img_box)
    pred = detector.predict(img)
    result_of_OCR = pred
    return result_of_OCR
  


In [ ]:
# get_info_word(img)[:-1]

In [ ]:
# for i in range(len(img_crop_list)):
#   if i % 5 == 4:
#     print(get_info_hand(img_crop_list[i]))
#     continue
#   print(get_info_word(img_crop_list[i]))
  
  

In [ ]:
len(img_crop_list)

# Thực hiện dự đoán 

In [ ]:
# import pandas as pd
# df = {"stt": ["0"], "mssv":["0"], "ho_va_ten":["0"], "lop":["0"], "diem":["0"] }
# df = pd.DataFrame(df)
# for i in range(5, len(img_crop_list), 5):
#   # if i > 4 :
#     # print(get_info_box(img_crop_list[i]))
#     # if i % 5 == 0:
#       df_sub = {}
#       df_sub["stt"] = [get_info_word(img_crop_list[i])]
#     # if i % 5 == 1:
#       df_sub["mssv"] = [get_info_word(img_crop_list[i+1])]
#     # if i % 5 == 2:
#       df_sub["ho_va_ten"] = [get_info_word(img_crop_list[i+2])]
#     # if i % 5 == 3:
#       df_sub["lop"] = [get_info_word(img_crop_list[i+3])]
#     # if i % 5 == 4 :
#       # print(get_info_hand(img_crop_list[i]))
#       df_sub["diem"] = [get_info_hand(img_crop_list[i+4])]
#       df_sub = pd.DataFrame(df_sub)
#       df = pd.concat([df,df_sub], ignore_index=True )   


In [ ]:
# df = df.drop([0])
# df

# Chỉnh nền 
_________________

In [ ]:
%%capture
!pip install skimage
!pip install imutils

In [ ]:
def preprocess_xa(img, factor: int):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = Image.fromarray(img)
    enhancer = ImageEnhance.Sharpness(img).enhance(factor)
    if gray.std() < factor:
        enhancer = ImageEnhance.Contrast(enhancer).enhance(factor)
    return np.array(enhancer)

In [ ]:
img_xa = img_crop_list[134]
# img_xa = preprocess_xa(img_xa, 20)
plot(img_xa)

# Sử dụng CRAFT


In [ ]:
# get_info_hand(img_xa)

In [ ]:
def boundingRect(bbox):
      bbox = bbox.astype(np.int)
      x0, y0 = np.min(bbox, axis=0)
      x1, y1 = np.max(bbox, axis=0)
      return x0, y0, x1, y1
detect_img_list = []
img_box = img_xa.copy()
result_of_OCR = ""
# img_box = img_crop_list[32]
text_boxes = craft.detect_text(image=img_box)['boxes']
if len(text_boxes) > 0:
    text_boxes = [boundingRect(bbox) for bbox in text_boxes]
    text_boxes = sorted(text_boxes, key=lambda x: (x[3], x[2]))
s = []
legit_box = []
list_detect = []
dict_of_OCR = {}
dict_of_det = {}
for bbox in text_boxes:
    x0, y0, x1, y1 = bbox
    if x0 < 0 or y0 < 0:
        continue
    try:
        img = img_box[y0 - 6: y1 + 6, x0  - 6: x1 + 6]
        detect_img_list.append(img)
        plot(img)
        img = Image.fromarray(img)
    
    except:
        try:
            img = img_box[y0 - 4: y1 + 4, x0  - 4: x1 + 4]
            detect_img_list.append(img)
            plot(img)
            img = Image.fromarray(img)
        except:
            try:
                img = img_box[y0 - 4: y1 + 4, x0  - 4: x1 + 4]
                detect_img_list.append(img)
                plot(img)
                img = Image.fromarray(img)
            except:
                img = img_box[y0: y1, x0: x1]
                detect_img_list.append(img)
                plot(img)
                img = Image.fromarray(img)


In [ ]:
# img_check = detect_img_list[0] 
# img_check = cv2.blur(img_check,(3,3))
# img_check = preprocess(img_check, 30)
# plot(img_check)
# gray_scale=cv2.cvtColor(img_check,cv2.COLOR_BGR2GRAY)
# th1,img_bin = cv2.threshold(gray_scale,50,200,cv2.THRESH_BINARY)
# img_bin=~img_bin
# plot(img_bin)
# img_bin = convert2Square(img_bin)
# # th1,img_bin = cv2.threshold(img_bin,150,200,cv2.THRESH_BINARY)
# plot(img_bin)


# CCA và Mnist

In [ ]:
from skimage.filters import threshold_local
from skimage import measure

from imutils import perspective
import imutils
import cv2
# from src.data_utils import order_points, convert2Square, draw_labels_and_boxes
def convert2Square(image):
    """
    Resize non square image(height != width to square one (height == width)
    :param image: input images
    :return: numpy array
    """

    img_h = image.shape[0]
    img_w = image.shape[1]

    # if height > width
    if img_h > img_w:
        diff = img_h - img_w
        if diff % 2 == 0:
            x1 = np.zeros(shape=(img_h, diff//2))
            x2 = x1
        else:
            x1 = np.zeros(shape=(img_h, diff//2))
            x2 = np.zeros(shape=(img_h, (diff//2) + 1))

        squared_image = np.concatenate((x1, image, x2), axis=1)
    elif img_w > img_h:
        diff = img_w - img_h
        if diff % 2 == 0:
            x1 = np.zeros(shape=(diff//2, img_w))
            x2 = x1
        else:
            x1 = np.zeros(shape=(diff//2, img_w))
            x2 = x1

        squared_image = np.concatenate((x1, image, x2), axis=0)
    else:
        squared_image = image

    squared_image = cv2.resize(squared_image, (28, 28))
    return squared_image


# apply thresh to extracted licences plate

def blur(LpRegion):
    V = cv2.split(cv2.cvtColor(LpRegion, cv2.COLOR_BGR2HSV))[2]

    # adaptive threshold
    T = threshold_local(V, 13, offset = 10, method="gaussian")
    thresh = (V > T).astype("uint8") * 255

    # convert black pixel of digits to white pixel
    thresh = cv2.bitwise_not(thresh)
    thresh = imutils.resize(thresh, width = 400)
    thresh = cv2.medianBlur(thresh, 5)

    # connected components analysis
    labels =    measure.label(thresh, connectivity=1, background=0)

    # plot(labels)



In [ ]:
def connected_components_analysis(img_check):
    img_check = cv2.blur(img_check,(3,3))
    img_check = preprocess(img_check, 30)
    # plot(img_check)
    gray_scale=cv2.cvtColor(img_check,cv2.COLOR_BGR2GRAY)
    img_bin=~gray_scale
    th1,img_bin = cv2.threshold(img_bin,70,251,cv2.THRESH_BINARY)
    # plot(img_bin)
    # img_bin = convert2Square(img_bin)
    th1,img_bin = cv2.threshold(img_bin,50,240,cv2.THRESH_BINARY)
    # plot(img_bin)
    # num_labels, labels_im = cv2.connectedComponents(img_bin)
    labels_im = measure.label(img_bin, connectivity=2, background=0)
    return img_bin, labels_im


In [ ]:
def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    plot(labeled_img)
img_bin, labels_im = connected_components_analysis(img_xa)
imshow_components(labels_im)

In [ ]:
def get_candidates_img(img_bin, labels_im ):
    candidate_list = []
    for label in np.unique(labels_im):
        # if this is background label, ignore it
        if label == 0:
            continue

        # init mask to store the location of the character candidates
        mask = np.zeros(img_bin.shape, dtype="uint8")
        mask[labels_im == label] = 255

        # find contours from mask
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            (x, y, w, h) = cv2.boundingRect(contour)

            # rule to determine characters
            aspectRatio = w / float(h)
            solidity = cv2.contourArea(contour) / float(w * h)
            heightRatio = h / float(labels_im.shape[0])

            if  solidity > 0.1 and 0.35 < heightRatio < 2.0:
            # if True:

                # extract characters
                candidate = np.array(mask[y:y + h, x:x + w])
                # plot(candidate)
                square_candidate = convert2Square(candidate)

                plot(square_candidate)
                
                candidate_list.append((square_candidate, x))

            # square_candidate = cv2.resize(square_candidate, (28, 28), cv2.INTER_AREA)
            # square_candidate = square_candidate.reshape((28, 28, 1))
            
    return candidate_list

# Mô hình dự đoán :


In [ ]:
import torch
from torch import nn
class Net(nn.Module):

    def __init__(self):
        super().__init__()

        # feature extractor
        self.activation = nn.Tanh()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        # classifer
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # feature extraction
        x = self.activation(self.conv1(x))
        x = self.avgpool(x)
        
        x = self.activation(self.conv2(x))
        x = self.avgpool(x)
        
        x = self.dropout1(x)
        
        # classification
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.dropout2(x)

        x = self.activation(self.fc2(x))
        x = self.dropout2(x)
        
        x = self.softmax(self.fc3(x))
        return x
model = Net()
model= torch.load("/content/drive/MyDrive/xa/code_xa/project3/weight_mnist/model_mnist_xa_drop.pt").cuda()

In [ ]:
import torchvision.transforms as transforms
def predict_score(candidates):
    pred_list = []
    result = 1000
    candidates = sorted(candidates, key = lambda x: x[1])
    transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize(
                                 (0.1307,), (0.3081,))
                                    
                                     ])
    for square_candidate, x in candidates:
        x  = square_candidate
        # ic(x)
        import scipy
        x = scipy.pad(array= x , pad_width=[5, 5], mode='constant', constant_values=0)
        x = cv2.resize(x, (28, 28))
        # plot(x)
        x = transform(x)

        # x = torch.tensor(x)
        x = torch.unsqueeze(x, dim=0 )
        # x = torch.unsqueeze(x, dim=0)
        x = x.type(torch.cuda.FloatTensor)


        x = x.cuda()
        pred =  model(x)
        # print("Dự đoán:")
        # print(pred)
        # ic("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        # print(torch.argmax(pred, dim=-1).cpu().detach().numpy())
        if pred.cpu().detach().numpy().max() > 0.55 :
            pred = str(torch.argmax(pred, dim=-1).cpu().detach().numpy()[0])
            pred_list.append(pred)

    try:
        result = (float(pred_list[0])*10 + float(pred_list[-1]))*1.0/10
    except:
        pass
    return "{:.1f}".format(result) 

# predict_score(candidate_list)

In [ ]:
def get_info_score(img_xa):
    img_bin, labels_im = connected_components_analysis(img_xa)
    candidate_list =  get_candidates_img(img_bin, labels_im )
     
    return predict_score(candidate_list)

In [ ]:
get_info_score(img_crop_list[34])

# MAIN

In [ ]:
import pandas as pd
from tqdm import tqdm
df = {"stt": ["0"], "mssv":["0"], "ho_va_ten":["0"], "lop":["0"], "diem":["0"] }
df = pd.DataFrame(df)
for i in range(5, len(img_crop_list), 5):
  # if i > 4 :
    # print(get_info_box(img_crop_list[i]))
    # if i % 5 == 0:
      df_sub = {}
      df_sub["stt"] = [get_info_word(img_crop_list[i])]
    # if i % 5 == 1:
      df_sub["mssv"] = [get_info_word(img_crop_list[i+1])]
    # if i % 5 == 2:
      df_sub["ho_va_ten"] = [get_info_word(img_crop_list[i+2])]
    # if i % 5 == 3:
      df_sub["lop"] = [get_info_word(img_crop_list[i+3])]
    # if i % 5 == 4 :
      # print(get_info_hand(img_crop_list[i]))
      df_sub["diem"] = [get_info_score(img_crop_list[i+4])]
      df_sub = pd.DataFrame(df_sub)
      df = pd.concat([df,df_sub], ignore_index=True )   
df

In [ ]:
for i in range(len(img_crop_list)):
    if i % 5 == 4 :
        print("--------------{i}-------------------------------".format(i = i))
        plot(img_crop_list[i])
        print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")

In [ ]:
df1 = df.drop([0])
df1.to_excel("/content/drive/MyDrive/xa/code_xa/project3/project3_data/qt2022.xlsx",index=False)